In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import platform
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import optuna
from sklearn.preprocessing import MinMaxScaler
import joblib

# 한글 폰트 설정
if platform.system() == "Darwin":  # 맥
    plt.rc("font", family="AppleGothic")
elif platform.system() == "Windows":  # 윈도우
    plt.rc("font", family="Malgun Gothic")
else:  # 리눅스 (우분투 등)
    plt.rc("font", family="NanumGothic")

plt.rcParams["axes.unicode_minus"] = False  # 마이너스 기호 깨짐 방지


# BASELINE 전처리
train_data_IVF = train_data[train_data["시술 유형"] == "IVF"]
train_data_IVF = train_data_IVF[
    train_data_IVF["배아 생성 주요 이유"].str.contains("현재 시술용", na=False)
]
train_data_IVF = train_data_IVF[train_data_IVF["이식된 배아 수"] != 0.0]
train_data_IVF = train_data_IVF.drop(
    columns=[
        "ID",
        "불임 원인 - 여성 요인",
        "시술 유형",
        "해동 난자 수",
        "저장된 신선 난자 수",
    ]
)

# 수치형 데이터 꼬리 짜르기
train_data_IVF.loc[train_data_IVF["총 생성 배아 수"] >= 22.0, "총 생성 배아 수"] = 22.0
train_data_IVF.loc[
    train_data_IVF["미세주입된 난자 수"] >= 22.0, "미세주입된 난자 수"
] = 22.0
train_data_IVF.loc[
    train_data_IVF["미세주입에서 생성된 배아 수"] >= 19.0, "미세주입에서 생성된 배아 수"
] = 19.0
train_data_IVF.loc[train_data_IVF["저장된 배아 수"] >= 11.0, "저장된 배아 수"] = 11.0
train_data_IVF.loc[
    train_data_IVF["미세주입 후 저장된 배아 수"] >= 1.0, "미세주입 후 저장된 배아 수"
] = 1.0
train_data_IVF.loc[train_data_IVF["해동된 배아 수"] >= 1.0, "해동된 배아 수"] = 1.0
train_data_IVF.loc[
    train_data_IVF["수집된 신선 난자 수"] >= 31.0, "수집된 신선 난자 수"
] = 31.0
train_data_IVF.loc[train_data_IVF["혼합된 난자 수"] >= 31.0, "혼합된 난자 수"] = 31.0
train_data_IVF.loc[
    train_data_IVF["파트너 정자와 혼합된 난자 수"] >= 31.0,
    "파트너 정자와 혼합된 난자 수",
] = 31.0
train_data_IVF.loc[
    train_data_IVF["기증자 정자와 혼합된 난자 수"] >= 23.0,
    "기증자 정자와 혼합된 난자 수",
] = 31.0

# test data 에서도 같은 전처리 적용
test_data_IVF = test_data[test_data["시술 유형"] == "IVF"]

test_data_IVF = test_data_IVF[
    test_data_IVF["배아 생성 주요 이유"].str.contains("현재 시술용", na=False)
]
test_data_IVF = test_data_IVF[test_data_IVF["이식된 배아 수"] != 0.0]
test_data_IVF = test_data_IVF.drop(
    columns=[
        "ID",
        "불임 원인 - 여성 요인",
        "시술 유형",
        "해동 난자 수",
        "저장된 신선 난자 수",
    ]
)
test_data_IVF.loc[test_data_IVF["총 생성 배아 수"] >= 22.0, "총 생성 배아 수"] = 22.0
test_data_IVF.loc[test_data_IVF["미세주입된 난자 수"] >= 22.0, "미세주입된 난자 수"] = (
    22.0
)
test_data_IVF.loc[
    test_data_IVF["미세주입에서 생성된 배아 수"] >= 19.0, "미세주입에서 생성된 배아 수"
] = 19.0
test_data_IVF.loc[test_data_IVF["저장된 배아 수"] >= 11.0, "저장된 배아 수"] = 11.0
test_data_IVF.loc[
    test_data_IVF["미세주입 후 저장된 배아 수"] >= 1.0, "미세주입 후 저장된 배아 수"
] = 1.0
test_data_IVF.loc[test_data_IVF["해동된 배아 수"] >= 1.0, "해동된 배아 수"] = 1.0
test_data_IVF.loc[
    test_data_IVF["수집된 신선 난자 수"] >= 31.0, "수집된 신선 난자 수"
] = 31.0
test_data_IVF.loc[test_data_IVF["혼합된 난자 수"] >= 31.0, "혼합된 난자 수"] = 31.0
test_data_IVF.loc[
    test_data_IVF["파트너 정자와 혼합된 난자 수"] >= 31.0,
    "파트너 정자와 혼합된 난자 수",
] = 31.0
test_data_IVF.loc[
    test_data_IVF["기증자 정자와 혼합된 난자 수"] >= 23.0,
    "기증자 정자와 혼합된 난자 수",
] = 31.0

categorical_cols = [
    "시술 시기 코드",
    "시술 당시 나이",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부",
    "임신 성공 여부",
]
numeric_cols = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    # "해동 난자 수",
    "수집된 신선 난자 수",
    # "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일",
]
avg_dicts = {
    "임신 시도 또는 마지막 임신 경과 연수": {
        0.0: 0.3333,
        1.0: 0.5,
        2.0: 0.1905,
        3.0: 0.2794,
        4.0: 0.2659,
        5.0: 0.2729,
        6.0: 0.2759,
        7.0: 0.2631,
        8.0: 0.2458,
        9.0: 0.2356,
        10.0: 0.2566,
        11.0: 0.2168,
        12.0: 0.2483,
        13.0: 0.1929,
        14.0: 0.2637,
        15.0: 0.2311,
        16.0: 0.1751,
        17.0: 0.2,
        18.0: 0.1809,
        19.0: 0.169,
        20.0: 0.2029,
        "nan": 0.3086,
    },
    "총 생성 배아 수": {
        0.0: 0.2473,
        1.0: 0.1114,
        2.0: 0.1977,
        3.0: 0.2628,
        4.0: 0.3003,
        5.0: 0.329,
        6.0: 0.3492,
        7.0: 0.3659,
        8.0: 0.383,
        9.0: 0.3882,
        10.0: 0.4009,
        11.0: 0.4122,
        12.0: 0.4032,
        13.0: 0.4272,
        14.0: 0.4323,
        15.0: 0.4387,
        16.0: 0.4466,
        17.0: 0.4389,
        18.0: 0.4455,
        19.0: 0.4598,
        20.0: 0.4586,
        21.0: 0.4436,
        22.0: 0.435,
    },
    "미세주입된 난자 수": {
        0.0: 0.2981,
        1.0: 0.104,
        2.0: 0.1473,
        3.0: 0.1975,
        4.0: 0.2374,
        5.0: 0.2816,
        6.0: 0.3036,
        7.0: 0.3213,
        8.0: 0.333,
        9.0: 0.3471,
        10.0: 0.365,
        11.0: 0.383,
        12.0: 0.3714,
        13.0: 0.3955,
        14.0: 0.3801,
        15.0: 0.3999,
        16.0: 0.3956,
        17.0: 0.4025,
        18.0: 0.3958,
        19.0: 0.4314,
        20.0: 0.4278,
        21.0: 0.4088,
        22.0: 0.4072,
    },
    "미세주입에서 생성된 배아 수": {
        0.0: 0.298,
        1.0: 0.1073,
        2.0: 0.1993,
        3.0: 0.2621,
        4.0: 0.3034,
        5.0: 0.3348,
        6.0: 0.3553,
        7.0: 0.3708,
        8.0: 0.3832,
        9.0: 0.396,
        10.0: 0.4103,
        11.0: 0.4278,
        12.0: 0.394,
        13.0: 0.4284,
        14.0: 0.4458,
        15.0: 0.4348,
        16.0: 0.4018,
        17.0: 0.4728,
        18.0: 0.4768,
        19.0: 0.4764,
    },
    "이식된 배아 수": {1.0: 0.3135, 2.0: 0.3111, 3.0: 0.1685},
    "미세주입 배아 이식 수": {0.0: 0.3036, 1.0: 0.3194, 2.0: 0.3123, 3.0: 0.1684},
    "저장된 배아 수": {
        0.0: 0.2516,
        1.0: 0.3856,
        2.0: 0.4221,
        3.0: 0.442,
        4.0: 0.453,
        5.0: 0.466,
        6.0: 0.4765,
        7.0: 0.4547,
        8.0: 0.4917,
        9.0: 0.4933,
        10.0: 0.4836,
        11.0: 0.4582,
    },
    "미세주입 후 저장된 배아 수": {0.0: 0.2806, 1.0: 0.4331},
    "해동된 배아 수": {0.0: 0.3188, 1.0: 0.2468},
    "수집된 신선 난자 수": {
        0.0: 0.2756,
        1.0: 0.1048,
        2.0: 0.1337,
        3.0: 0.1717,
        4.0: 0.2114,
        5.0: 0.2424,
        6.0: 0.2572,
        7.0: 0.2821,
        8.0: 0.3059,
        9.0: 0.311,
        10.0: 0.3343,
        11.0: 0.3485,
        12.0: 0.362,
        13.0: 0.3673,
        14.0: 0.3661,
        15.0: 0.3763,
        16.0: 0.3771,
        17.0: 0.3889,
        18.0: 0.4029,
        19.0: 0.4057,
        20.0: 0.3949,
        21.0: 0.4092,
        22.0: 0.3927,
        23.0: 0.412,
        24.0: 0.4231,
        25.0: 0.4217,
        26.0: 0.3852,
        27.0: 0.3862,
        28.0: 0.401,
        29.0: 0.3456,
        30.0: 0.4025,
        31.0: 0.3759,
    },
    "혼합된 난자 수": {
        0.0: 0.2474,
        1.0: 0.1097,
        2.0: 0.1434,
        3.0: 0.195,
        4.0: 0.2341,
        5.0: 0.2776,
        6.0: 0.2914,
        7.0: 0.312,
        8.0: 0.3288,
        9.0: 0.339,
        10.0: 0.3567,
        11.0: 0.3685,
        12.0: 0.3704,
        13.0: 0.3867,
        14.0: 0.3772,
        15.0: 0.3853,
        16.0: 0.396,
        17.0: 0.4012,
        18.0: 0.4081,
        19.0: 0.4083,
        20.0: 0.4125,
        21.0: 0.4112,
        22.0: 0.4065,
        23.0: 0.4364,
        24.0: 0.4483,
        25.0: 0.3894,
        26.0: 0.399,
        27.0: 0.3612,
        28.0: 0.3782,
        29.0: 0.4023,
        30.0: 0.4015,
        31.0: 0.3664,
    },
    "파트너 정자와 혼합된 난자 수": {
        0.0: 0.2691,
        1.0: 0.1108,
        2.0: 0.1437,
        3.0: 0.1949,
        4.0: 0.2317,
        5.0: 0.2767,
        6.0: 0.2898,
        7.0: 0.3091,
        8.0: 0.3276,
        9.0: 0.3368,
        10.0: 0.3542,
        11.0: 0.3692,
        12.0: 0.3693,
        13.0: 0.3852,
        14.0: 0.3751,
        15.0: 0.384,
        16.0: 0.392,
        17.0: 0.3999,
        18.0: 0.4076,
        19.0: 0.4094,
        20.0: 0.4078,
        21.0: 0.4119,
        22.0: 0.3978,
        23.0: 0.4367,
        24.0: 0.4494,
        25.0: 0.3896,
        26.0: 0.405,
        27.0: 0.3531,
        28.0: 0.3846,
        29.0: 0.391,
        30.0: 0.3968,
        31.0: 0.3697,
    },
    "기증자 정자와 혼합된 난자 수": {
        0.0: 0.3051,
        1.0: 0.1045,
        2.0: 0.1456,
        3.0: 0.1959,
        4.0: 0.2604,
        5.0: 0.2907,
        6.0: 0.3124,
        7.0: 0.3398,
        8.0: 0.3393,
        9.0: 0.3645,
        10.0: 0.3891,
        11.0: 0.361,
        12.0: 0.3831,
        13.0: 0.406,
        14.0: 0.3995,
        15.0: 0.4083,
        16.0: 0.4397,
        17.0: 0.411,
        18.0: 0.4242,
        19.0: 0.3934,
        20.0: 0.459,
        21.0: 0.4066,
        22.0: 0.4783,
        31.0: 0.433,
    },
    "난자 채취 경과일": {0.0: 0.3149, "nan": 0.2756},
    "난자 해동 경과일": {0.0: 0.2673, 1.0: 0.0, "nan": 0.3065},
    "난자 혼합 경과일": {
        0.0: 0.3189,
        1.0: 0.2066,
        2.0: 0.2688,
        3.0: 0.3167,
        4.0: 0.5,
        5.0: 0.2632,
        6.0: 0.0,
        7.0: 0.0,
        "nan": 0.2482,
    },
    "배아 이식 경과일": {
        0.0: 0.251,
        1.0: 0.187,
        2.0: 0.2125,
        3.0: 0.2588,
        4.0: 0.3444,
        5.0: 0.4045,
        6.0: 0.2998,
        7.0: 0.4111,
        "nan": 0.2662,
    },
    "배아 해동 경과일": {
        0.0: 0.2473,
        1.0: 0.2,
        2.0: 0.1737,
        3.0: 0.1765,
        4.0: 0.0,
        5.0: 0.2941,
        6.0: 0.2778,
        "nan": 0.3188,
    },
    "시술 시기 코드": {
        "TRCMWS": 0.3223,
        "TRDQAZ": 0.3178,
        "TRJXFG": 0.308,
        "TRVNRY": 0.2982,
        "TRXQMD": 0.289,
        "TRYBLT": 0.3202,
        "TRZKPL": 0.2891,
    },
    "시술 당시 나이": {
        "만18-34세": 0.3764,
        "만35-37세": 0.324,
        "만38-39세": 0.256,
        "만40-42세": 0.1938,
        "만43-44세": 0.1529,
        "만45-50세": 0.219,
    },
    "특정 시술 유형": {
        "FER": 0.5,
        "ICSI": 0.3087,
        "ICSI / AH": 0.2215,
        "ICSI / AH:Unknown": 1.0,
        "ICSI / BLASTOCYST ": 0.3691,
        "ICSI / BLASTOCYST :ICSI": 1.0,
        "ICSI / BLASTOCYST :IVF / BLASTOCYST": 0.25,
        "ICSI / BLASTOCYST:IVF / BLASTOCYST": 0.3333,
        "ICSI:ICSI": 0.1864,
        "ICSI:IVF": 0.3445,
        "ICSI:Unknown": 0.1458,
        "IVF": 0.318,
        "IVF / AH": 0.1885,
        "IVF / AH:ICSI / AH": 0.0,
        "IVF / BLASTOCYST": 0.3758,
        "IVF:ICSI": 0.3841,
        "IVF:IVF": 0.2703,
        "IVF:Unknown": 0.2464,
        "Unknown": 0.2567,
    },
    "배란 자극 여부": {0: 0.2695, 1: 0.3172},
    "배란 유도 유형": {
        "기록되지 않은 시행": 0.3172,
        "생식선 자극 호르몬": 1.0,
        "세트로타이드 (억제제)": 0.0,
        "알 수 없음": 0.2695,
    },
    "단일 배아 이식 여부": {0.0: 0.2833, 1.0: 0.3672},
    "착상 전 유전 검사 사용 여부": {1.0: 0.319, "nan": 0.3062},
    "착상 전 유전 진단 사용 여부": {0.0: 0.3059, 1.0: 0.3504},
    "남성 주 불임 원인": {0: 0.308, 1: 0.2484},
    "남성 부 불임 원인": {0: 0.3073, 1: 0.2271},
    "여성 주 불임 원인": {0: 0.3081, 1: 0.2496},
    "여성 부 불임 원인": {0: 0.3073, 1: 0.2241},
    "부부 주 불임 원인": {0: 0.3082, 1: 0.2491},
    "부부 부 불임 원인": {0: 0.3071, 1: 0.2151},
    "불명확 불임 원인": {0: 0.308, 1: 0.3011},
    "불임 원인 - 난관 질환": {0: 0.309, 1: 0.2904},
    "불임 원인 - 남성 요인": {0: 0.2974, 1: 0.3205},
    "불임 원인 - 배란 장애": {0: 0.3021, 1: 0.3333},
    "불임 원인 - 자궁경부 문제": {0: 0.3063, 1: 0.0},
    "불임 원인 - 자궁내막증": {0: 0.3074, 1: 0.2923},
    "불임 원인 - 정자 농도": {0: 0.3062, 1: 0.2976},
    "불임 원인 - 정자 면역학적 요인": {0: 0.3062, 1: 0.0},
    "불임 원인 - 정자 운동성": {0: 0.3063, 1: 0.2048},
    "불임 원인 - 정자 형태": {0: 0.3063, 1: 0.2689},
    "배아 생성 주요 이유": {
        "기증용, 배아 저장용, 현재 시술용": 0.35,
        "기증용, 현재 시술용": 0.4276,
        "난자 저장용, 현재 시술용": 0.2,
        "배아 저장용, 현재 시술용": 0.3333,
        "현재 시술용": 0.3043,
    },
    "총 시술 횟수": {
        "0회": 0.3478,
        "1회": 0.2902,
        "2회": 0.2846,
        "3회": 0.2806,
        "4회": 0.2696,
        "5회": 0.2592,
        "6회 이상": 0.2509,
    },
    "클리닉 내 총 시술 횟수": {
        "0회": 0.3379,
        "1회": 0.2854,
        "2회": 0.2782,
        "3회": 0.2756,
        "4회": 0.2647,
        "5회": 0.2534,
        "6회 이상": 0.24,
    },
    "IVF 시술 횟수": {
        "0회": 0.3477,
        "1회": 0.2908,
        "2회": 0.2845,
        "3회": 0.276,
        "4회": 0.2664,
        "5회": 0.2506,
        "6회 이상": 0.2383,
    },
    "DI 시술 횟수": {
        "0회": 0.3061,
        "1회": 0.2886,
        "2회": 0.3135,
        "3회": 0.3279,
        "4회": 0.3225,
        "5회": 0.3035,
        "6회 이상": 0.3042,
    },
    "총 임신 횟수": {
        "0회": 0.3075,
        "1회": 0.3026,
        "2회": 0.2937,
        "3회": 0.2857,
        "4회": 0.2469,
        "5회": 0.2,
        "6회 이상": 0.5,
    },
    "IVF 임신 횟수": {
        "0회": 0.3076,
        "1회": 0.3025,
        "2회": 0.2899,
        "3회": 0.2813,
        "4회": 0.25,
        "5회": 0.2,
        "6회 이상": 1.0,
    },
    "DI 임신 횟수": {
        "0회": 0.3061,
        "1회": 0.3405,
        "2회": 0.4167,
        "3회": 0.3,
        "4회": 0.0,
        "5회": 0.0,
    },
    "총 출산 횟수": {
        "0회": 0.3063,
        "1회": 0.3064,
        "2회": 0.2978,
        "3회": 0.3235,
        "4회": 0.2308,
        "5회": 0.0,
        "6회 이상": 0.0,
    },
    "IVF 출산 횟수": {
        "0회": 0.3064,
        "1회": 0.306,
        "2회": 0.2916,
        "3회": 0.3254,
        "4회": 0.2308,
        "5회": 0.0,
    },
    "DI 출산 횟수": {
        "0회": 0.3061,
        "1회": 0.3564,
        "2회": 0.3902,
        "3회": 0.5,
        "5회": 0.0,
    },
    "난자 출처": {"기증 제공": 0.3476, "본인 제공": 0.3033},
    "정자 출처": {"기증 제공": 0.3167, "미할당": 0.2281, "배우자 제공": 0.3053},
    "난자 기증자 나이": {
        "만20세 이하": 0.2969,
        "만21-25세": 0.3665,
        "만26-30세": 0.3834,
        "만31-35세": 0.3368,
        "알 수 없음": 0.303,
    },
    "정자 기증자 나이": {
        "만20세 이하": 0.2765,
        "만21-25세": 0.3169,
        "만26-30세": 0.3259,
        "만31-35세": 0.3185,
        "만36-40세": 0.3185,
        "만41-45세": 0.3108,
        "알 수 없음": 0.3053,
    },
    "동결 배아 사용 여부": {0.0: 0.3188, 1.0: 0.2469},
    "신선 배아 사용 여부": {0.0: 0.2472, 1.0: 0.3187},
    "기증 배아 사용 여부": {0.0: 0.3058, 1.0: 0.349},
    "대리모 여부": {0.0: 0.3062, 1.0: 0.3058},
    "PGD 시술 여부": {1.0: 0.3516, "nan": 0.3059},
    "PGS 시술 여부": {1.0: 0.3194, "nan": 0.3062},
    "임신 성공 여부": {0: 0.0, 1: 1.0},
}


# %%
def min_max_scale_fixed_min_max(data, fixed_min=0.4, fixed_max=0.7):
    # 고정된 min, max를 사용하여 Min-Max Scaling
    scaled_data = (data - fixed_min) / (fixed_max - fixed_min)
    return np.clip(scaled_data, 0, 1)  # 0과 1 사이로 클리핑


def process_data(data_IVF, avg_dicts, is_train=True):
    X = []  # CNN의 입력 데이터
    y = []  # 정답 데이터 (train인 경우만 사용)

    for i in range(len(data_IVF)):
        print(f"Processing sample {i} {'(train)' if is_train else '(test)'}")

        # 현재 샘플 가져오기
        a = data_IVF.iloc[i]
        pix = []

        # 각 열 이름과 값을 이용해 pix 생성
        for n, v in a.items():
            if pd.isna(v):
                v = "nan"
            pix.append(avg_dicts[n][v])

        if is_train:
            # 정답 레이블 추가
            y.append(pix[63])  # 예시로 pix[63]을 정답으로 사용

        # 첫 63개의 값을 사용
        pix = pix[:63]

        # 2. 63 x 63 크기의 행렬 생성
        matrix_size = 63
        matrix = np.zeros((matrix_size, matrix_size))

        # (m, n) 위치 값 계산
        for m in range(matrix_size):
            for n in range(m, matrix_size):
                value = pix[m] + pix[n]
                matrix[m, n] = value
                matrix[n, m] = value

        # Min-Max Scaling
        matrix = min_max_scale_fixed_min_max(matrix)

        # 3. CNN 입력 데이터로 추가
        X.append(matrix)

    # CNN 입력 데이터로 변환 및 채널 추가
    X = np.array(X).reshape((len(X), 63, 63, 1)).astype("float32")

    if is_train:
        y = np.array(y)
        return X, y
    else:
        return X


# Train 데이터 처리
X_train, y_train = process_data(train_data_IVF, avg_dicts, is_train=True)
joblib.dump((X_train, y_train), "train.pkl")
print("Train 데이터가 train.pkl로 저장되었습니다.")

# Test 데이터 처리
X_test = process_data(test_data_IVF, avg_dicts, is_train=False)
joblib.dump(X_test, "test.pkl")
print("Test 데이터가 test.pkl로 저장되었습니다.")


# %%
# 전처리된 이미지 데이터 시각화
import matplotlib.pyplot as plt

l = [
    "시술 시기 코드",
    "시술 당시 나이",
    "임신 시도 또는 마지막 임신 경과 연수",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "수집된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일",
    "임신 성공 여부",
]

for i in range(30):
    plt.figure(figsize=(30, 30))
    plt.imshow(X_train[i], cmap="viridis")
    plt.colorbar(label="Pixel Value")

    plt.xticks(ticks=range(len(l)), labels=l, fontsize=18, rotation=90)
    plt.yticks(ticks=range(len(l)), labels=l, fontsize=18)

    plt.title(f"y_train: {y_train[i]}, index: {i}", fontsize=25)
    plt.axis("on")
    plt.show()


In [ ]:
import joblib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

# 1. 저장된 데이터 불러오기 (원본 X_train의 shape: (N, 63, 63, 1))
X_train, _ = joblib.load("train.pkl")  # 예: (42640, 63, 63, 1)

# 2. 이미지 리사이즈: (63,63,1) -> (32,32,1)
X_train_resized = tf.image.resize(X_train, (32, 32))
X_train_resized = np.array(X_train_resized).astype("float32")

# Define latent dimension
latent_dim = 8

# Encoder
inputs = keras.Input(shape=(32, 32, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(latent_dim, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)

# 샘플링 함수
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])
encoder = keras.Model(inputs, [z_mean, z_log_var, z], name="encoder")

# Decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(8 * 8 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((8, 8, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, x, name="decoder")

# VAE Model
outputs = decoder(z)
vae = keras.Model(inputs, outputs, name="vae")

# ✅ KL 손실을 모델에 추가하는 방법 수정 (Lambda Layer 사용)
class KLLossLayer(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        self.add_loss(K.mean(kl_loss))
        return inputs  # 원본 입력값을 그대로 반환

# ✅ KL 손실을 모델 내부에 추가
kl_layer = KLLossLayer()([z_mean, z_log_var])

# ✅ Reconstruction Loss 추가
vae.compile(optimizer=keras.optimizers.Adam(), loss="binary_crossentropy")

# Train model (리사이즈된 X_train_resized 사용)
vae.fit(X_train_resized, X_train_resized, epochs=5, batch_size=128, validation_split=0.1)

# Visualizing original and reconstructed images
def plot_images(original, reconstructed, num_images=10):
    plt.figure(figsize=(20, 4))
    for i in range(num_images):
        # Original images
        ax = plt.subplot(2, num_images, i + 1)
        plt.imshow(original[i].squeeze(), cmap="gray")
        plt.axis("off")
        
        # Reconstructed images
        ax = plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(reconstructed[i].squeeze(), cmap="gray")
        plt.axis("off")
    plt.show()

# Generate reconstructed images
x_decoded = vae.predict(X_train_resized[:10])
plot_images(X_train_resized[:10], x_decoded)

# VAE 인코딩된 CNN 특징 벡터 추출 및 저장 (학습 데이터)
z_mean_train, _, _ = encoder.predict(X_train_resized)
joblib.dump(z_mean_train, "z_mean_train_dim8.pkl")
print("✅ VAE 인코딩된 특징 벡터가 'z_mean_train_dim8.pkl'로 저장되었습니다.")

# ---------------------------------------------
# 테스트 데이터에 대해서도 동일하게 수행
X_test = joblib.load("test.pkl")  # (M, 63, 63, 1)
X_test_resized = tf.image.resize(X_test, (32, 32))
X_test_resized = np.array(X_test_resized).astype("float32")
z_mean_test, _, _ = encoder.predict(X_test_resized)
joblib.dump(z_mean_test, "z_mean_test_dim8.pkl")
print("✅ 테스트 데이터에 대한 VAE 인코딩된 특징 벡터가 'z_mean_test_dim8.pkl'로 저장되었습니다.")


In [ ]:
import numpy as np
import pandas as pd
import joblib
import optuna
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

# ✅ 1️⃣ 데이터 로드 및 필터링
# train_data는 이미 로드되어 있다고 가정 (예: pd.read_csv() 등)
train = train_data.drop(columns=["ID"])
train = train[train["시술 유형"] == "IVF"]

test = pd.read_csv("test.csv").drop(columns=["ID"])
test = test[test["시술 유형"] == "IVF"]

X = train.drop("임신 성공 여부", axis=1)
y = train["임신 성공 여부"]

# ✅ 2️⃣ VAE 인코딩된 CNN 특징 벡터 불러오기
z_mean_train = joblib.load("z_mean_train_dim8.pkl")  # (N, 32) 크기; N 예: 42640
z_mean_test = joblib.load("z_mean_test_dim8.pkl")  # (N, 32) 크기

# ✅ 3️⃣ 차원 축소 (여기서는 UMAP을 사용하여 6차원으로 축소)
umap_reducer = umap.UMAP(n_components=6, random_state=42)
z_umap_train = umap_reducer.fit_transform(z_mean_train)
z_umap_test = umap_reducer.transform(z_mean_test)
z_pca_train = z_umap_train  # 학습 데이터 변환
z_pca_test = z_umap_test    # 테스트 데이터 변환

print(f"✅ 변환 완료! 기존 차원: {z_mean_train.shape[1]}, 축소된 차원: {z_pca_train.shape[1]}")

# ✅ 4️⃣ 범주형 및 수치형 변수 정의
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",   
    "특정 시술 유형",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "PGS 시술 여부",
]

numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "파트너 정자와 혼합된 난자 수",
    "난자 혼합 경과일",
    "배아 이식 경과일",
]

l = categorical_columns + numeric_columns
print(len(l))

X = X[l]
test = test[l]

# ✅ 5️⃣ 결측치 처리: 수치형 변수는 중앙값으로 대체
imputer = SimpleImputer(strategy="median")
X[numeric_columns] = imputer.fit_transform(X[numeric_columns])
test[numeric_columns] = imputer.transform(test[numeric_columns])

# ✅ 6️⃣ 범주형 변수 처리: 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

# ★★ 중요한 수정 ★★
# PCA 특징 벡터(z_pca_train)는 학습 데이터의 일부 샘플(예: 42640개)만 포함합니다.
# 따라서 기존 데이터(X)와 레이블(y)도 동일한 샘플 수로 슬라이싱합니다.
X = X.iloc[: z_pca_train.shape[0]]
y = y.iloc[: z_pca_train.shape[0]]

# ✅ 7️⃣ PCA 특징 벡터 추가 (CNN + 기존 데이터 결합)
# X.values의 샘플 수를 PCA 벡터와 동일하게 슬라이싱
X_subset = X.values  # 이미 슬라이싱 되었으므로 X.values의 행 수 == z_pca_train.shape[0]
X_combined = np.hstack([z_pca_train, X_subset])  # 학습 데이터 결합

# 테스트 데이터 결합
# 만약 테스트 데이터와 VAE 테스트 벡터의 샘플 수가 다르다면 동일하게 슬라이싱합니다.
test = test.iloc[: z_pca_test.shape[0]]
test_subset = test.values
test_combined = np.hstack([z_pca_test, test_subset])

# ✅ 8️⃣ CatBoost 범주형 변수 인덱스 설정
cat_features = list(
    range(z_pca_train.shape[1], z_pca_train.shape[1] + len(categorical_columns))
)

# ✅ 9️⃣ Optuna를 사용한 CatBoost 하이퍼파라미터 튜닝
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 500, 1000),
        "depth": trial.suggest_int("depth", 6, 9),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.05, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 2.0, 4.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.5, 1.0),
        "random_strength": trial.suggest_float("random_strength", 9.0, 10.0),
        "random_seed": 42,
        "verbose": 0,
        "cat_features": cat_features,
    }

    X_train_split, X_val, y_train_split, y_val = train_test_split(
        X_combined, y, test_size=0.2, random_state=42
    )

    model = CatBoostClassifier(**params)
    model.fit(
        X_train_split, y_train_split, eval_set=(X_val, y_val),
        early_stopping_rounds=50, verbose=0
    )

    val_preds = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, val_preds)

# ✅ 🔟 Optuna 최적화 수행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# ✅ 1️⃣1️⃣ 최적의 하이퍼파라미터 적용 후 모델 학습
best_params = study.best_params
final_model = CatBoostClassifier(
    **best_params, cat_features=cat_features, random_seed=42
)

final_model.fit(X_combined, y)

# ✅ 1️⃣2️⃣ 테스트 데이터 예측
pred_proba = final_model.predict_proba(test_combined)[:, 1]

# ✅ 1️⃣ PCA 시각화 (2D 변환)
from sklearn.decomposition import PCA
pca_2d = PCA(n_components=2)
z_pca_2d = pca_2d.fit_transform(z_mean_train)

plt.figure(figsize=(8, 6))
plt.scatter(z_pca_2d[:, 0], z_pca_2d[:, 1], c=y, cmap="viridis", alpha=0.7)
plt.colorbar(label="Label")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("PCA Visualization of CNN Features", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

# ✅ 4️⃣ Feature Importance 시각화 (한글 깨짐 없이 정렬 유지)
import platform
pca_feature_names = [f"PCA_Feature_{i}" for i in range(z_pca_train.shape[1])]
original_feature_names = categorical_columns + numeric_columns
all_feature_names = pca_feature_names + original_feature_names
feature_importances = final_model.get_feature_importance()

if platform.system() == "Darwin":  # MacOS
    plt.rc("font", family="AppleGothic")
plt.rcParams["axes.unicode_minus"] = False

plt.figure(figsize=(12, 25))
sorted_indices = np.argsort(feature_importances)[::-1]
sorted_feature_names = np.array(all_feature_names)[sorted_indices]
sorted_importances = feature_importances[sorted_indices]

plt.barh(sorted_feature_names, sorted_importances, color="royalblue", alpha=0.8)
plt.xlabel("Feature Importance Score", fontsize=13)
plt.ylabel("Feature Name", fontsize=13)
plt.title("CatBoost Feature Importance", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis="x", linestyle="--", alpha=0.6)
plt.show()

# ✅ 추가 전처리 완료 및 테스트 데이터 예측 부분
test_combined = np.hstack([z_pca_test, test.values])
print(f"✅ 테스트 데이터 전처리 완료! 최종 Shape: {test_combined.shape}")

pred_proba = final_model.predict_proba(test_combined)[:, 1]

# 테스트 데이터와 제출 파일 불러오기
test_data = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

# test_data의 ID와 pred_proba를 딕셔너리로 매핑
pred_dict = dict(zip(test_data["ID"], pred_proba))

# submission의 "ID"를 기준으로 매핑하여 "probability" 컬럼 채우기 (매칭 안되는 값은 NaN -> 0.0으로 대체)
submission["probability"] = submission["ID"].map(pred_dict).fillna(0.0)

submission.to_csv("submission_cat_vae_25_32to6.csv", index=False)
print("최종 제출 파일이 'submission_cat_vae_25_32to6.csv'로 저장되었습니다.")
